In [1]:
!pip install llama-index openai PyMuPDF pandas
!pip install tqdm



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install transformers

     ---------------------------------------- 0.0/43.6 kB ? eta -:--:--
     ----------------- -------------------- 20.5/43.6 kB 682.7 kB/s eta 0:00:01
     -------------------------------------- 43.6/43.6 kB 538.5 kB/s eta 0:00:00
  Using cached tokenizers-0.19.1-cp311-none-win_amd64.whl.metadata (6.9 kB)
   ---------------------------------------- 0.0/9.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.3 MB 1.9 MB/s eta 0:00:05
   - -------------------------------------- 0.3/9.3 MB 3.9 MB/s eta 0:00:03
   -- ------------------------------------- 0.6/9.3 MB 5.8 MB/s eta 0:00:02
   ------- -------------------------------- 1.7/9.3 MB 12.2 MB/s eta 0:00:01
   ------------- -------------------------- 3.1/9.3 MB 16.2 MB/s eta 0:00:01
   ---------------- ----------------------- 3.8/9.3 MB 17.3 MB/s eta 0:00:01
   -------------------- ------------------- 4.7/9.3 MB 17.6 MB/s eta 0:00:01
   ----------------------- ---------------- 5.6/9.3 MB 17.8 MB/s eta 0:00:01
   -------


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import fitz  # PyMuPDF
import pandas as pd
import os
import random
from transformers import T5Tokenizer, T5ForConditionalGeneration
from tqdm import tqdm
import time

c:\Users\engba\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
txt_path = "C:/Users/engba/Desktop/merged_output.txt"

In [6]:
import subprocess

try:
    with open(txt_path, 'r', encoding='utf-8') as file:
        for i in range(20):
            line = file.readline()
            if line.strip():
                print(line.strip())
except FileNotFoundError:
    print(f"could not find: {txt_path}")
except Exception as e:
    print(f"error: {e}")

Neutrinos & Non-proliferation in Europe
Michel Cribier*
APC, Paris
CEA/Saclay, DAPNIA/SPP
The International Atomic Energy Agency (IAEA) is the United Nations agency in
charge of the development of peaceful use of atomic energy. In particular IAEA is the
verification authority of the Treaty on the Non-Proliferation of Nuclear Weapons (NPT). To
do that jobs inspections of civil nuclear installations and related facilities under safeguards
agreements are made in more than 140 states.
IAEA uses many different tools for these verifications, like neutron monitor, gamma
spectroscopy, but also bookeeping of the isotopic composition at the fuel element level before
and after their use in the nuclear power station. In particular it verifie that weapon-origin and
other fissile materials that Russia and USA have released from their defense programmes are
used for civil application.
The existence of an antineutrino signal sensitive to the power and to the isotopic
composition of a reactor core, as 

In [8]:
# T5 modeli ve tokenizer'ı yükle
model_name = "ramsrigouthamg/t5_squad_v1"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

In [9]:
def generate_questions_from_text_file(txt_path, tokenizer, model, num_questions=70000, max_length=512):
    with open(txt_path, 'r', encoding='utf-8') as f:
        text = f.read()

    questions = []
    text_length = len(text)
    
    start_time = time.time()
    for i in tqdm(range(num_questions), desc="Generating Questions"):
        # Rastgele başlangıç noktası seçme
        if text_length > max_length:
            start_idx = random.randint(0, text_length - max_length)
        else:
            start_idx = 0

        input_text = text[start_idx:start_idx + max_length]

        # Girdi oluşturma
        input_text = f"generate question: {input_text}"
        if not input_text.endswith("</s>"):
            input_text += " </s>"

        # Soru üretme
        input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=max_length, truncation=True)
        output_ids = model.generate(input_ids, max_length=64, num_beams=4, early_stopping=True)
        question = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        # "question:" önekini kaldırma
        question = question.replace("question:", "").strip()

        questions.append(question)

        # Anlık olarak soruyu yazdırma
        print(f"Question {i+1}: {question}")

    return questions

# Soru üretme
questions = generate_questions_from_text_file(txt_path, tokenizer, model, num_questions=70000)

# DataFrame oluşturma ve CSV dosyasına yazma
df = pd.DataFrame({"question": questions})
df.to_csv("/content/questions.csv", index=False)

print("CSV file created: questions.csv")


Generating Questions:   0%|          | 0/70000 [00:00<?, ?it/s]c:\Users\engba\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\t5\tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
Generating Questions:   0%|          | 1/70000 [00:03<68:52:12,  3.54s/it]

Question 1: What type of lung inflammations were observed in rodent inhalation studies?


c:\Users\engba\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\t5\tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
Generating Questions:   0%|          | 2/70000 [00:05<49:55:45,  2.57s/it]

Question 2: What part of the reactor was leaking light water from a crack in the reflector system?


Generating Questions:   0%|          | 3/70000 [00:07<42:36:11,  2.19s/it]

...

Question 5: What is a Coefficient of expansion?


Generating Questions:   0%|          | 6/70000 [00:13<36:49:00,  1.89s/it]

Question 6: Where is the University of New Mexico?


Generating Questions:   0%|          | 6/70000 [00:13<43:45:22,  2.25s/it]


KeyboardInterrupt: 